In [1]:
pip install import_ipynb

  Stored in directory: C:\Users\shenti10\AppData\Local\pip\Cache\wheels\b4\7b\e9\a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Stemming and Lemmatisation
from datetime import date, timedelta
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words = set(stopwords.words("english"))
new_words = ['not_the']
stop_words = stop_words.union(new_words)
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
import statistics as stats
import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import import_ipynb
import utils.Io
import utils.Matrix_m
import utils.Timesplit
import utils.Key_phrase

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


importing Jupyter notebook from W:\Github\iNAGO\iNAGO_RecSys\Yelp_data_exploration\utils\Io.ipynb
importing Jupyter notebook from W:\Github\iNAGO\iNAGO_RecSys\Yelp_data_exploration\utils\Matrix_m.ipynb
importing Jupyter notebook from W:\Github\iNAGO\iNAGO_RecSys\Yelp_data_exploration\utils\Timesplit.ipynb
importing Jupyter notebook from W:\Github\iNAGO\iNAGO_RecSys\Yelp_data_exploration\utils\Key_phrase.ipynb


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Models

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    """
    get the item-keyphrase matrix
    """
    rows = []
    cols = []
    vals = []
    #For each review history
    for i in tqdm(range(X.shape[0])):
        #Get the array of frequencies for document/review i 
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line, keyphrase that appears index 
        length_of_nonzero = len(nonzero_element) #number of important keyphrase that appears
        
        # df[row_name][i] is the item idex
        #Get a list row index that indicates the document/review
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        #print(rows)
        
        #Get a list of column index indicating the key phrase that appears in i document/review
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            #Create a bunch of 1s
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            #If not binary 
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res

#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

#topK: the number of restuarants we are suggesting 
#if vector_train has number, then the user has visited
def sub_routine(vector_u, vector_train, topK=500):

    #index where non-zero
    train_index = vector_train.nonzero()[1]
    
    vector_u = vector_u
    
    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u[:topK]

## Evaluation

In [4]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        #"R-Precision": r_precision,
        #"NDCG": ndcg,
        #"Clicks": click
    }

    local_metrics = {
        #"Precision": precisionk,
        #"Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))
        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = round(results[name],4)
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (round((np.average(results[name])),4),
                                                              round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))
    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = round(results[name],4)
    else:
        for name in global_metric_names:
            results_summary[name] = (round(np.average(results[name]),4), round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
    output.update(results_summary)

    return output

### TuneK method

In [5]:
def individualKNNPrediction (similarityMatrix, predictionMatrix, kRange, validOrTestMatrix):
    "Declaration for kRange = range(50,120,10)"
    similarity = train(similarityMatrix)
    MAP10 = {}
    for kValue in kRange:
        user_item_prediction_score = predict(predictionMatrix, kValue, similarity, item_similarity_en= False)
        user_item_predict = prediction(user_item_prediction_score, 50, predictionMatrix)
        user_item_res = evaluate(user_item_predict, validOrTestMatrix)
        
        MAP10[kValue] = user_item_res.get('MAP@10')[0]
        
    return MAP10

def simpleKNNPrediction (similarityMatrix, predictionMatrix, kValue, validOrTestMatrix):
    "Declaration for kRange = range(50,120,10)"
    similarity = train(similarityMatrix)
    user_item_prediction_score = predict(predictionMatrix, kValue, similarity, item_similarity_en= False)
    user_item_predict = prediction(user_item_prediction_score, 50, predictionMatrix)
    user_item_res = evaluate(user_item_predict, validOrTestMatrix)

    MAP10= user_item_res.get('MAP@10')[0]
        
    return MAP10

## Dataset files 

In [7]:
#reviewJson = "..\\data\\Export_CleanedReview.json"
reviewJsonWithClosedRes = "..\\data\\Export_CleanedReviewWithClosedRes.json"
reviewJsonWithBusAvgStars = "..\\data\\Export_CleanedReviewNew_1011.json"
# with open(reviewJson, encoding = 'utf-8') as json_file:
#     data = json_file.readlines()
#     data = list(map(json.loads, data))

#This appeared to be a neasted list, just use data[0] to read
# df_review = pd.DataFrame(data[0])

In [8]:
# 2013 - 2018
cross_dict = {}
for year in range(2013,2018):
    temp = {}
#     df = utils.Io.get_yelp_df(path ='', filename=reviewJsonWithBusAvgStars, sampling= True, cross_valid = True, start_date = '2012-01-01', end_date = '2013-06-01')
    df = utils.Io.get_yelp_df(path ='', filename=reviewJsonWithBusAvgStars, sampling= True, cross_valid = True, start_date = str(year) + '-01-01', end_date = str(year+1) + '-06-01')

    df['reviewStars_userAvg'] = df['review_stars'] - df['user_avg_stars']
    df.loc[df['reviewStars_userAvg'] == 0, "reviewStars_userAvg"] = 0.01
    df['reviewStars_busAvg'] = df['review_stars'] - df['business_stars']
    df.loc[df['reviewStars_busAvg'] == 0, "reviewStars_busAvg"] = 0.01

    rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, IC_Matrix = utils.Matrix_m.get_rating_timestamp_matrix(df)

    rtrain, rvalid, rtest, rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, rtime, item_idx_matrix_train,item_idx_matrix_valid, item_idx_matrix_test = utils.Timesplit.time_ordered_split(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=False)  
    rtrain_implicit, rvalid_implicit, rtest_implicit, rtrain_userAvg_implicit, rvalid_userAvg_implicit, rtest_userAvg_implicit, nonzero_index, rtime, item_idx_matrix_train_implicit,item_idx_matrix_valid_implicit, item_idx_matrix_test_implicit = utils.Timesplit.time_ordered_split(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=True)
    U_C_matrix_explicit,U_C_matrix_implicit = utils.Matrix_m.get_UC_Matrix(IC_Matrix,rtrain_implicit)
    userVisitMatrix = utils.Matrix_m.getImplicitMatrix(rtrain)

    
    kRange = range(50,120,10)
    
    #Get the MAP list for the current sampling size

    mapList1 = individualKNNPrediction(rtrain, rtrain, kRange, rvalid)
#     maximum = max(mapList1, key=mapList1.get)
#     k1 = mapList1[maximum]
    temp['combination1'] = mapList1
    
    mapList2 = individualKNNPrediction(rtrain_userAvg, rtrain_userAvg, kRange, rvalid)
#     maximum = max(mapList2, key=mapList2.get)
#     k2 = mapList2[maximum]
    temp['combination2'] = mapList2
    
    mapList3 = individualKNNPrediction(userVisitMatrix, userVisitMatrix, kRange, rvalid)
#     maximum = max(mapList3, key=mapList3.get)
#     k3 = mapList3[maximum]
    temp['combination3'] = mapList3
    
    mapList4 = individualKNNPrediction(rtrain_implicit, rtrain_implicit, kRange, rvalid)
#     maximum = max(mapList4, key=mapList4.get)
#     k4 = mapList4[maximum]
    temp['combination4'] = mapList4
    
    mapList5 = individualKNNPrediction(U_C_matrix_explicit, userVisitMatrix, kRange, rvalid)
#     maximum = max(mapList5, key=mapList5.get)
#     k5 = mapList5[maximum]
    temp['combination5'] = mapList5
    
    cross_dict[year] = temp
        

100%|████████████████████████████████████████████████████████████████████████████| 6003/6003 [00:01<00:00, 3944.34it/s]
W:\Capstone\Tina\iNAGO_RecSys\Yelp_data_exploration\utils\Timesplit.ipynb:11: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  "from os.path import isfile, join\n",
100%|████████████████████████████████████████████████████████████████████████████| 6003/6003 [00:01<00:00, 5211.29it/s]
W:\Capstone\Tina\iNAGO_RecSys\Yelp_data_exploration\utils\Matrix_m.ipynb:8: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  "source": [
100%|████████████████████████████████████████████████████████████████████████████| 6003/6003 [00:01<00:00, 4955.81it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6003/6003 [00:01<00:00, 4829.04it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6011/6011 [00:00<00:00, 9806.56it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6011/6011 [00:00<00:00, 9809.46it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6022/6022 [00:00<00:00, 7530.09it/s]


...

100%|████████████████████████████████████████████████████████████████████████████| 6007/6007 [00:00<00:00, 7565.37it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6007/6007 [00:00<00:00, 7491.62it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6034/6034 [00:01<00:00, 4682.37it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6034/6034 [00:01<00:00, 4567.02it/s]


100%|████████████████████████████████████████████████████████████████████████████| 6034/6034 [00:00<00:00, 9276.61it/s]


In [9]:
cross_dict

{2013: {'combination1': {110: 0.0183},
  'combination2': {100: 0.0128},
  'combination3': {110: 0.0184},
  'combination4': {100: 0.0141},
  'combination5': {50: 0.017}},
 2014: {'combination1': {110: 0.0201},
  'combination2': {100: 0.0104},
  'combination3': {110: 0.0208},
  'combination4': {90: 0.0161},
  'combination5': {50: 0.016}},
 2015: {'combination1': {100: 0.0233},
  'combination2': {110: 0.0127},
  'combination3': {80: 0.0229},
  'combination4': {110: 0.018},
  'combination5': {50: 0.0177}},
 2016: {'combination1': {110: 0.0215},
  'combination2': {100: 0.0147},
  'combination3': {70: 0.022},
  'combination4': {100: 0.0168},
  'combination5': {50: 0.019}},
 2017: {'combination1': {110: 0.0251},
  'combination2': {100: 0.0149},
  'combination3': {110: 0.0251},
  'combination4': {110: 0.0205},
  'combination5': {50: 0.0174}}}

In [ ]:
kRange = range(50,120,10)
#Get the MAP list for the current sampling size
mapList = individualKNNPrediction(rtrain_implicit, rtrain, kRange, rvalid)
plotMAPList = mapList

In [ ]:
maximum = max(mapList, key=mapList.get)
k1 = mapList[maximum]
k1

In [ ]:
maximum = max(mydict, key=mydict.get)